# NeuroMANCER Node and System classes and modules tutorial

This script demonstrates how to use NeuroMANCER Node to wrap arbitrary callable
into symbolic representation that can be used in NeuroMANCER problem formulation.


### Install (Colab only)
Skip this step when running locally.

We need to install a more recent version of matplotlib than is offered in the default Colab environment. After running the cell 1 for the first time in a new Colab runtime, you will see the prompt: "You must restart the runtime in order to use newly installed versions." After restarting, the correct version of matplotlib will be loaded for the duration of the current runtime.

In [ ]:
!pip install --upgrade matplotlib

In [ ]:
!pip install setuptools==61.0.0 casadi mlflow torchdiffeq dill pyts plum-dispatch==1.7.3 --user
!pip install git+https://github.com/pnnl/neuromancer.git@master --ignore-requires-python --no-deps --user

### Import

In [1]:
import torch
from neuromancer.system import Node, System

## Node

**Node** is a simple class to create symbolic modules out of arbitrary PyTorch callables.
Node class is wrapping the callable and defines the computational node based 
on input_keys and output_keys that define computational node connections through 
intermediate dictionaries. Complex symbolic architectures can be constructed by connecting
input and output keys of a set of Nodes via System and Problem classes.
   

In [2]:
# 1, wrap nn.Linear into Node
net_1 = torch.nn.Linear(1, 1)
node_1 = Node(net_1, ['x1'], ['y1'])
# print input and output keys
print(node_1.input_keys)
print(node_1.output_keys)
# evaluate forward pass of the node with dictionary input dataset
print(node_1({'x1': torch.rand(1)}))

['x1']
['y1']
{'y1': tensor([0.3844], grad_fn=<AddBackward0>)}


In [3]:
# 2, wrap nn.Sequential into Node
net_2 = torch.nn.Sequential(torch.nn.Linear(2, 5),
                            torch.nn.ReLU(),
                            torch.nn.Linear(5, 3),
                            torch.nn.ReLU(),
                            torch.nn.Linear(3, 1))
node_2 = Node(net_2, ['x2'], ['y2'])
# evaluate forward pass of the node with dictionary input dataset
print(node_2({'x2': torch.rand(2)}))

{'y2': tensor([-0.1138], grad_fn=<AddBackward0>)}


In [4]:
# 3, wrap arbitrary callable into Node - allows for unwrapping the inputs
fun_1 = lambda x1, x2: 2.*x1 - x2**2
node_3 = Node(fun_1, ['y1', 'y2'], ['y3'], name='quadratic')
# evaluate forward pass of the node with dictionary input dataset
print(node_3({'y1': torch.rand(2), 'y2': torch.rand(2)}))

{'y3': tensor([0.7336, 0.4167])}


In [5]:
# 4, wrap callable with multiple inputs and outputs
def fun_2(x1, x2):
    return x1**2, x2**2
node_4 = Node(fun_2, ['x1', 'x2'], ['x1^2', 'x2^2'], name='square')
# evaluate forward pass of the node with dictionary input dataset
print(node_4({'x1': torch.rand(2), 'x2': torch.rand(2)}))

{'x1^2': tensor([0.1574, 0.0009]), 'x2^2': tensor([0.1470, 0.3897])}


## Modules

NeuroMANCER also provides implementation of useful building blocks for
creating custom neural architectures. These include:
* modules.blocks          - neural architecures
* modules.activations     - custom activation functions    
* modules.functions       - useful callables 
* modules.gnn             - graph neural nets
* modules.rnn             - recurent neural nets
* modules.solvers         - iterative solvers for constrained optimization
* slim.linear             - linear algebra factorizations for weights
        
Next set of example shows how to wrap NeuroMANCER modules into Node

In [6]:
from neuromancer.modules import blocks
from neuromancer.modules import activations
from neuromancer import slim

In [10]:
# for a full list of available blocks (nn.Modules) in NeuroMANCER see:
print(blocks.blocks)
# for a full list of available activations in NeuroMANCER see:
print(activations.activations)

{'mlp': <class 'neuromancer.modules.blocks.MLP'>, 'rnn': <class 'neuromancer.modules.blocks.RNN'>, 'pytorch_rnn': <class 'neuromancer.modules.blocks.PytorchRNN'>, 'linear': <class 'neuromancer.modules.blocks.Linear'>, 'residual_mlp': <class 'neuromancer.modules.blocks.ResMLP'>, 'basislinear': <class 'neuromancer.modules.blocks.BasisLinear'>, 'bilinear': <class 'neuromancer.modules.blocks.BilinearTorch'>, 'icnn': <class 'neuromancer.modules.blocks.InputConvexNN'>, 'pos_def': <class 'neuromancer.modules.blocks.PosDef'>}
{'softexp': <class 'neuromancer.modules.activations.SoftExponential'>, 'blu': <class 'neuromancer.modules.activations.BLU'>, 'aplu': <class 'neuromancer.modules.activations.APLU'>, 'prelu': <class 'neuromancer.modules.activations.PReLU'>, 'pelu': <class 'neuromancer.modules.activations.PELU'>, 'relu': <class 'torch.nn.modules.activation.ReLU'>, 'gelu': <class 'torch.nn.modules.activation.GELU'>, 'rrelu': <class 'torch.nn.modules.activation.RReLU'>, 'hardtanh': <class 'tor

In [8]:
# 1, instantiate 4-layer multilayer perceptron with linear weight and ReLU activation
block_1 = blocks.MLP(insize=2, outsize=3,
                  bias=True,
                  linear_map=slim.maps['linear'],
                  nonlin=torch.nn.ReLU,
                  hsizes=[80] * 4)
# wrap modules into Node
node_4 = Node(block_1, ['x3'], ['y3'])
# evaluate forward pass of the node with dictionary input dataset
data = {'x3': torch.rand(10, 2)}
print(node_4(data).keys())
print(node_4(data)['y3'].shape)

dict_keys(['y3'])
torch.Size([10, 3])


In [9]:
# 2, instantiate recurrent neural net without bias, SVD linear map, and BLI activation
block_2 = blocks.RNN(insize=2, outsize=2,
                  bias=False,
                  linear_map=slim.linear.SVDLinear,
                  nonlin=activations.BLU,
                  hsizes=[80] * 4)
# wrap modules into Node
node_5 = Node(block_2, ['x4'], ['y4'])
# evaluate forward pass of the node with dictionary input dataset
data = {'x4': torch.rand(10, 2)}
print(node_5(data).keys())
print(node_5(data)['y4'].shape)

dict_keys(['y4'])
torch.Size([10, 2])


## System

**System** is a class that supports construction of cyclic computational graphs in NeuroMANCER.
System's graph is defined by a list of Nodes. Instantiated System can be used to simulate
dynamical systems in open or closed loop rollouts by specifying number of steps via nsteps.

In [11]:
# 1, create acyclic symbolic graph
# list of nodes to construct the graph
nodes = [node_1, node_2, node_3]
# 10 steps rollout
nsteps = 3
# connecting nodes via System class
system_1 = System(nodes, nsteps=nsteps)
# print input and output keys
print(system_1.input_keys)
print(system_1.output_keys)
# evaluate forward pass of the System with 3D input dataset
batch = 2
print(system_1({'x1': torch.rand(batch, nsteps, 1),
                'x2': torch.rand(batch, nsteps, 2)}))

['x2', 'x1']
['y1', 'y2', 'y3']
{'x1': tensor([[[0.9535],
         [0.0577],
         [0.0512]],

        [[0.6492],
         [0.7798],
         [0.0388]]]), 'x2': tensor([[[0.8419, 0.8008],
         [0.1785, 0.8557],
         [0.6865, 0.0123]],

        [[0.6075, 0.2561],
         [0.4420, 0.5080],
         [0.0563, 0.0185]]]), 'y1': tensor([[[0.9154],
         [0.3005],
         [0.2960]],

        [[0.7065],
         [0.7962],
         [0.2875]]], grad_fn=<CatBackward0>), 'y2': tensor([[[-0.0655],
         [-0.0992],
         [-0.0919]],

        [[-0.0950],
         [-0.0968],
         [-0.1138]]], grad_fn=<CatBackward0>), 'y3': tensor([[[1.8266],
         [0.5911],
         [0.5836]],

        [[1.4041],
         [1.5830],
         [0.5620]]], grad_fn=<CatBackward0>)}


In [ ]:
# visualize symbolic computational graph
# system_1.show()

In [12]:
# 2, close the loop by creating recursion in one of the nodes
nodes[2].output_keys = ['y1']
# create new system with cyclic computational graph
system_2 = System(nodes, nsteps=nsteps)
# print input and output keys
print(system_2.input_keys)
print(system_2.output_keys)
# evaluate forward pass of the System with 3D input dataset
print(system_1({'x1': torch.rand(batch, nsteps, 1),
                'x2': torch.rand(batch, nsteps, 2)}))

['x2', 'x1']
['y1', 'y2']
{'x1': tensor([[[0.5867],
         [0.9016],
         [0.8294]],

        [[0.6909],
         [0.6105],
         [0.3444]]]), 'x2': tensor([[[0.2761, 0.5576],
         [0.1684, 0.1988],
         [0.0133, 0.8928]],

        [[0.7608, 0.3762],
         [0.4422, 0.3508],
         [0.3585, 0.3125]]]), 'y1': tensor([[[0.6636],
         [1.3165],
         [0.8798],
         [2.6199],
         [0.8302],
         [1.7487]],

        [[0.7351],
         [1.4634],
         [0.6799],
         [2.9163],
         [0.4972],
         [1.3482]]], grad_fn=<CatBackward0>), 'y2': tensor([[[-0.1040],
         [-0.1138],
         [-0.1048]],

        [[-0.0828],
         [-0.1021],
         [-0.1078]]], grad_fn=<CatBackward0>)}


In [ ]:
# visualize symbolic computational graph
# system_1.show()